For Reference (single and multi qubit tomography): https://research.physics.illinois.edu/QI/Photonics/tomography/

Importing the necassary libraries

In [52]:
import numpy as np
from itertools import product
import re

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit.library import *
from qiskit.primitives import StatevectorSampler

Function to do complete tomography on a state generated by a ciruit.

In [59]:
def construct_state(circuit, samples):

    # Get the number of qubits in the circuit.
    num_qubits = len(circuit.qubits)

    # The density matrix of an n qubit state can be represented as a linear combination of tensor products of the Pauli matrices.
    paulis = [
        [[1,0],
         [0,1]],
        [[0,1],
         [1,0]],
        [[0,-1j],
         [1j,0]],
        [[1,0],
         [0,-1]]
    ]

    # Create a sampler to simulate the quantum circuit on a single shot.
    sampler = StatevectorSampler(default_shots=samples)

    # Generate all the measurment vectors. (Sets of tensor products of Pauli matrices)
    measurement_vectors = product([0,1,2,3], repeat=num_qubits)

    for m in measurement_vectors:
        # Add gates to the circuit to measure the state in the basis defined by the measurement vector.
        meas_circuit = circuit.copy()
        for i in m:
            match i:
                case 0:
                    continue
                case 1:
                    meas_circuit.h(i)
                case 2:
                    meas_circuit.h(i)
                    meas_circuit.sdg(i)
                case 3:
                    continue

        meas_circuit.measure_all()

        # Run the sampler on the quantum circuit and store the result.
        result = sampler.run([meas_circuit]).result()[0]

        # Store the bitstrings representing the measured basis states. Because the sampler is configered to run the circuit once, there is only one result.
        counts = result.data.meas.get_counts()   

        # Get the single bitstring.
        states = list(list(counts.keys())[0])
        # The bit string is ordered with qubit n on the left and qubit 0 on the right. We want to reverse this order.
        states.reverse()

        # Make the substrings to search for.
        substring = ''
        for i in m:
            match i:
                case 0:
                    substring += '.'
                case 1:
                    substring += '1'
                case 2:
                    substring += '1'
                case 3:
                    substring += '1'
    

    # Calculate all the multi-qubit stokes parameters.
    


Using the useful fact that the trace of A tensor B is the product of the traces of A and B.